In [1]:
# %load blncq.py
import pygame
import torch

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import random
import matplotlib.pyplot as plt
import os
from collections import deque

import IPython.display

print("OK")

pygame.init()

# Pygame screen size:
sx = 900
sy = 500

# Table hgitht, left brim, right brim...
tableH = sy-20
tableL = int(sx*0.1)
tableR = int(sx*0.9)

# Stick picture size:
stickL = 400
stickW = 50

cticker = pygame.time.Clock()
testsc = pygame.display.set_mode((sx, sy))

im_bg = pygame.image.load("assets/bg.png").convert_alpha()
im_st = pygame.image.load("assets/pp.png").convert_alpha()

im_bg = pygame.transform.scale(im_bg, (sx,sy))
im_st = pygame.transform.scale(im_st, (stickW, stickL))

# 
def get_bottom(img, th, bwidth = stickW):
    x = -img.get_rect()[2]
    y = -img.get_rect()[3]
    if np.sin(th)<0:
        x=0
    if np.cos(th)<0:
        y=0
    bx = np.sign(np.cos(th)) * np.abs(np.sin(th)) * bwidth/2
    by = np.sign(np.sin(th)) * np.abs(np.cos(th)) * bwidth/2
    return (x+by, y+bx)

#for i in range(1300):
#     testsc.blit(im_bg, (0,0))
#     tst = pygame.transform.rotate(im_st, i)
#     x,y = get_bottom(tst, i*np.pi/180)
#     testsc.blit(tst, (500+x,300+y))
#     # testsc.blit(im_bd, (300,500))
#     # print(tst.get_rect(),(x,y))
#     pygame.display.update()
#     cticker.tick(30)
    


class gameState():
    def __init__(self, th,w, x, v, g=1, zhuanguan=10):
        self.th = th
        self.w = w
        self.x = x
        self.v = v
        self.g = g
        self.guanliang = zhuanguan

    def forward(self, dv, disp = 0):
        self.v += dv
        self.x += self.v
        self.w +=  ( dv * np.cos(self.th*np.pi/180) + self.g * np.sin(self.th*np.pi/180))/self.guanliang
        self.th += self.w
        
        if disp == 1:
            one_frame = self.drawgame()
        else:
            one_frame = []

        ##
        stateLabel = 1
        if self.x < tableL or self.x > tableR or self.th < -90 or self.th > 90:
            stateLabel = 0
        ##
        return np.stack((self.th, self.w, self.x, self.v), axis=0), stateLabel, one_frame
    
    def drawgame(self):
        testsc.blit(im_bg, (0, 0))
        tst = pygame.transform.rotate(im_st, self.th)
        x,y = get_bottom(tst, self.th*np.pi/180)
        testsc.blit(tst, (self.x + x, tableH + y))
        pygame.display.update()
        cticker.tick(30)
        return pygame.surfarray.array3d(pygame.display.get_surface())

    def printgame(self):
        print("Theta:%s, w: %s, position_x: %s, v: %s" % (self.th, self.w, self.x, self.v))
    ##

    def plotgame(self, fig, img):
        IPython.display.clear_output(wait=True)
        img.set_data(image_data)
        IPython.display.display(fig)
        cticker.tick(30)

    def getState(self):
        return np.stack((self.th, self.w, self.x, self.v) , axis=0)

    def gameinit(self, dis = 5):
        self.th = (np.random.rand()-0.5)*2*dis
        self.w = 0.0
        self.x = np.random.choice(range(tableL+200, tableR-200))
        self.v = np.random.rand()-0.5
        

game = gameState(1.0,0.0,400.0,0.0)

# for i in range(200):
#     x,s = game.forward(np.random.rand()-0.5,1)
#     # game.printgame()
#     if s==0:
#         game.printgame()
#         break

    


OK


In [2]:
class Blcer(nn.Module):
    def __init__(self, inputN, outputN, hiddenN):
        super(Blcer, self).__init__()
        self.inputN = inputN
        self.outputN = outputN
        self.hiddenN = hiddenN

        self.fc1 = nn.Linear(inputN, hiddenN)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hiddenN, outputN)
    ##
    def forward(self, input):
        out = self.fc1(input)
        out = self.relu(out)
        out = self.fc2(out)
        return out

vdlist = np.arange(19)-9


try:
    blcer = torch.load("models/naewBalancer.mdl")
except:
    print("No such model file to load")
    blcer = Blcer(4,19,10)
   


crit = nn.MSELoss()

optimizer = torch.optim.Adam( blcer.parameters(), lr=1e-3 )


def xbut01(x,a):
    rt = np.zeros(len(x))
    rt[a] = 1
    return rt

nTrain = 10240000
nPlaySteps = 400
nOptSteps = 50
nReplayMem = 10000
batchSize = 40

delta = 0.99
D = deque()

lossCurve = []

No such model file to load


In [ ]:
episilon = 0.1
em = 0.0
en = 1.0
for epoch in range(nTrain):
    epochLoss = 0
    for t in range(nPlaySteps):
        
        s_t = game.getState()
        q_sa = blcer(Variable(torch.FloatTensor(s_t)))
        a_t = torch.max(q_sa,0)[1].data[0]
        if np.random.rand()<episilon*en+em:
            a_t = np.random.randint(len(vdlist))
            en = en/(en+1)
        vd = vdlist[a_t]
        s_tt, l_tt, one_frame = game.forward(vd)
        if l_tt == 0:
            r_t = -100
            game.gameinit(0)
        else:
            r_t = 1 # +np.exp(-vd**2)

        D.append((s_t, a_t, r_t, s_tt, l_tt))
        if len(D) > nReplayMem:
            D.popleft()
        ##
        #if np.random.rand()<0.001:
        #    game.gameinit(10)
            
        
    
    ##
    for t in range(nOptSteps):
        optimizer.zero_grad()
        minibatch = random.sample(D, batchSize)
        
#        s_tb = [i[0] for i in minibatch]
#        a_tb = [i[1] for i in minibatch]
#        r_tb = [i[2] for i in minibatch]
#        s_ttb = [i[3] for i in minibatch]
#        sl_ttb = [i[4] for i in minibatch]
        
        states = np.stack([sample_in_batch for sample_in_batch in minibatch], axis = 1)

        s_ttb = Variable(torch.FloatTensor(states[3,:]))
        s_tb = Variable(torch.FloatTensor(states[0,:]))
        
        
        q_tt = blcer(s_ttb)
        q_tt = torch.max(q_tt, 1)[0]
        
        for i in range(len(q_tt)):
            q_tt[i] = q_tt[i] * states[4,i]
        ##
        q_better = []
        for i in range(batchSize):
            if states[4,i]==1:
                qnew = delta * q_tt[i].data[0] + states[2,i]
            else:
                qnew = states[2,i]
            q_better.append(qnew)
        q_better = Variable(torch.FloatTensor(q_better))
        
        
        q_t = blcer(s_tb)
        
        bad_indexes = []
        #Variable(torch.Tensor(np.array([xbut01(q_t[i],a_tb[i]) for i in range(len(a_tb))])))
        for i in range(len(q_t)):
            oneindex = xbut01(vdlist,states[1,i])
            bad_indexes.append(oneindex)
        ##
        bad_indexes = Variable(torch.FloatTensor(bad_indexes))
        
        q_bad = q_t.mul(bad_indexes).sum(1)
        
        loss = crit(q_bad, q_better)
        epochLoss += loss
        
        loss.backward()
        optimizer.step()
       
    ##
    if epoch%100 == 1:
        print("During Epoch %s we found the loss to be: %s" %  ( epoch, epochLoss.data[0]/nOptSteps))
        torch.save(blcer, "../models/newBalancer.mdl")
    lossCurve.append(epochLoss.data[0]/nOptSteps)                
plt.plot(lossCurve)
plt.show()

During Epoch 1 we found the loss to be: 98.1825


/home/me/anaconda3/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Blcer. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
plt.plot(lossCurve)
plt.gca().set_xlabel("%s-timeTraining round"%(nOptSteps))
plt.gca().set_ylabel("Loss")
plt.show()

In [ ]:
print(list(blcer.parameters()))

In [3]:
try:
    blcer2 = torch.load("models/usableBalancer.mdl")
except:
    print("No such model file to load...")

fig = plt.figure()
axe = fig.add_subplot(111)
dat = np.zeros((sy, sx))
img = axe.imshow(dat)

def illusnet(game,blcmdl):
    stat = game.getState()
    # print(stat)
    stat = Variable(torch.Tensor(stat))
    q_s = blcmdl(stat)
    # print(torch.max(q_s,0))
    a = torch.max(q_s,0)[1].data[0]
    q_action = q_s.data[a]
    vd = vdlist[a]
    # print(stat.data.numpy(),vd)
    stat, label, one_frame= game.forward(vd,disp=1)
    if label == 0:
        game.gameinit(dis = 10)
    ##
    IPython.display.clear_output(wait = True)
    img.set_data(np.transpose(one_frame,(1,0,2)))
    img.autoscale()
    axe.set_title("$\hat{Q}$ = %s"%(q_action))
    IPython.display.display(fig)
##

game.gameinit(40)
for i in range(2000):
    illusnet(game, blcer2)
    if i%300 == 1:
        game.gameinit(dis=20)

KeyboardInterrupt: 

In [93]:
# %load advancednn
class ActionNN(nn.Module):
    def __init__(self, ni, nh=10):
        super(ActionNN, self).__init__()
        self.inputN = ni
        self.hiddenN = nh
        self.fc1 = nn.Linear(ni, nh)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(nh, 1)
    
    ##
    def forward(self, input):
        out = self.fc1(input)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    ##
##

class QValue(nn.Module):
    def __init__(self, ni, nh=10):
        super(QValue, self).__init__()
        self.inputN = ni
        self.fc1 = nn.Linear(ni, nh)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(nh,1)
    ##
    def farward(self, stateAction):
        sa = stateAction
        out = self.fc1(sa)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    ##
##

netA = ActionNN(4)
netQ = QValue(5)
critS = nn.MSELoss()
optimizerA = torch.optim.Adam(netA.parameters(), lr=1e-3)
optimizerS = torch.optim.Adam(netQ.parameters(), lr = 1e-3)

records = deque()
recoMax = 50000

def catsa(s,a):
    return torch.cat([torch.Tensor(s), torch.Tensor(a)])

game.gameinit()
def repeatGame(g, ann, epochN=100):
    for epoch in range(epochN):
        s1 = g.getState()
        a = ann(Variable(torch.FloatTensor(s1)))
        
        s2, label, oneframe = g.forward(a.data[0])
        if label == 0:
            r = -100
            g.gameinit()
        else:
            r = 1
        epochRecord = (s1, a, r, s2, label)
        records.append(epochRecord)
        if len(records) > recoMax:
            records.popleft()
##
repeatGame(game,netA)

In [31]:
epochN = 1000
batchSize = 40
stepQ = 30
stepA = 30

for epoch in range(epochN):
    if epoch % 100 == 0:
        repeatGame(game, netA)

    for epoch in range(stepQ):
        minibatch = random.sample(records, batchSize)
        qt = [ netQ(catsa(m[0], m[1])) for m in minibatch ]
        qtt =[ netQ(catsa(m[3], netA(m[3]))) * m[4] for m in minibatch ]
        rt = [m[2] for m in minibatch]

    minibatch = random.sample(records, batchSize)
    qt = []
    qtt = []
    
    for m in minimatch:
        qtsa = netQ(catsa(m[0], netS(m[0]))) 
        qttsa = netQ(catsa(m[3], netS))
    lossA = -(torch.sum([netQ(netS(catsa(m[0], m[1])) for m in minibatch]))
    qt = [netQ(m[0]) for m in minibatch]
    qtt = []
    for i in range(batchSize):
        asd = minibatch[i][4] * netQ(minibatch[i][3])
        asd.requires_grad = False
        qtt.append(asd)
    ##
    rt = [m[2]]
    lossQ = 
    
    

[(array([ -4.79224543e+00,   0.00000000e+00,   5.55000000e+02,
        -2.16496611e-01]), -3159.49169921875, -100, array([ -319.64526674,  -314.85302131, -2604.70819583, -3159.70819583]), 0), (array([ -8.51211017e-02,   0.00000000e+00,   3.10000000e+02,
         2.56301800e-01]), -1764.478759765625, -100, array([ -176.53295092,  -176.44782982, -1454.22245797, -1764.22245797]), 0), (array([   2.80957264,    0.        ,  377.        ,    0.45139495]), -2146.084716796875, -100, array([ -211.53602981,  -214.34560245, -1768.63332185, -2145.63332185]), 0), (array([ -1.32872139e+00,   0.00000000e+00,   6.05000000e+02,
        -2.37898341e-01]), -3443.200439453125, -100, array([ -345.55850029,  -344.2297789 , -2838.43833779, -3443.43833779]), 0), (array([  2.95581381e+00,   0.00000000e+00,   3.60000000e+02,
        -2.80484384e-01]), -2046.1834716796875, -100, array([ -201.38515232,  -204.34096613, -1686.46395606, -2046.46395606]), 0), (array([ -2.71531249e+00,   0.00000000e+00,   4.77000000e+

[(array([ -3.70954132e+00,   0.00000000e+00,   5.32000000e+02,
        -3.60040420e-02]), -3028.9052734375, -100, array([ -305.97193986,  -302.26239854, -2496.94127748, -3028.94127748]), 0), (array([  3.44478496e+00,   0.00000000e+00,   4.41000000e+02,
         7.66438393e-02]), -2508.912109375, -100, array([ -246.98709866,  -250.43188362, -2067.83546554, -2508.83546554]), 0), (array([   3.68517066,    0.        ,  294.        ,    0.32982777]), -1672.536376953125, -100, array([ -163.21620714,  -166.9013778 , -1378.20654918, -1672.20654918]), 0), (array([ -2.02640578e+00,   0.00000000e+00,   4.62000000e+02,
         2.16825395e-01]), -2630.71728515625, -100, array([ -264.93715493,  -262.91074915, -2168.50045976, -2630.50045976]), 0), (array([  1.41318163e+00,   0.00000000e+00,   4.80000000e+02,
        -3.16790161e-01]), -2730.03759765625, -100, array([ -271.50507575,  -272.91825738, -2250.35438782, -2730.35438782]), 0), (array([   3.99122054,    0.        ,  311.        ,   -0.4011179

[(array([ -4.97033519e-01,   0.00000000e+00,   5.60000000e+02,
        -3.44622540e-01]), -3186.18408203125, -100, array([ -319.10432072,  -318.6072872 , -2626.52870457, -3186.52870457]), 0), (array([  3.67898812e+00,   0.00000000e+00,   3.43000000e+02,
        -2.64996995e-01]), -1949.198974609375, -100, array([ -190.83280508,  -194.51179319, -1606.4639716 , -1949.4639716 ]), 0), (array([  1.78896208e+00,   0.00000000e+00,   4.52000000e+02,
         2.41200819e-01]), -2572.73583984375, -100, array([ -255.35610315,  -257.14506523, -2120.49463902, -2572.49463902]), 0), (array([ -9.99287198e-02,   0.00000000e+00,   5.23000000e+02,
        -7.90810346e-02]), -2976.40185546875, -100, array([ -297.73983599,  -297.63990727, -2453.4809365 , -2976.4809365 ]), 0), (array([  1.85842398e+00,   0.00000000e+00,   4.66000000e+02,
        -1.07697803e-01]), -2651.022705078125, -100, array([ -263.10116278,  -264.95958676, -2185.13040288, -2651.13040288]), 0), (array([ -2.42790143e+00,   0.00000000e+00

[(array([  1.26480785e+00,   0.00000000e+00,   3.26000000e+02,
         2.45623848e-01]), -1855.178955078125, -100, array([ -184.20567995,  -185.47048781, -1528.93333123, -1854.93333123]), 0), (array([  4.05239288e+00,   0.00000000e+00,   5.86000000e+02,
        -4.12058114e-01]), -3332.673828125, -100, array([ -328.37470325,  -332.42709614, -2747.08588624, -3333.08588624]), 0), (array([  -1.75005818,    0.        ,  309.        ,    0.34050208]), -1759.61865234375, -100, array([ -177.63290169,  -175.8828435 , -1450.27815026, -1759.27815026]), 0), (array([ -1.98739989e+00,   0.00000000e+00,   5.09000000e+02,
         3.78548030e-01]), -2899.09765625, -100, array([ -291.7262463 ,  -289.73884641, -2389.71910822, -2898.71910822]), 0), (array([ -3.22038235e+00,   0.00000000e+00,   4.58000000e+02,
         7.52277492e-02]), -2607.69775390625, -100, array([ -263.58397903,  -260.36359667, -2149.62252616, -2607.62252616]), 0), (array([  6.67706747e-02,   0.00000000e+00,   5.14000000e+02,
     

[(array([  -0.97058434,    0.        ,  352.        ,   -0.45398969]), -2001.049072265625, -100, array([ -201.04847512,  -200.07789077, -1649.50306195, -2001.50306195]), 0), (array([ -6.59659412e-01,   0.00000000e+00,   3.69000000e+02,
         2.61107170e-01]), -2100.746826171875, -100, array([ -210.72157032,  -210.0619109 , -1731.485719  , -2100.485719  ]), 0), (array([  6.89089639e-01,   0.00000000e+00,   5.51000000e+02,
         8.07054816e-02]), -3136.325439453125, -100, array([ -312.91956909,  -313.60865873, -2585.24473397, -3136.24473397]), 0), (array([  2.51132932e+00,   0.00000000e+00,   5.83000000e+02,
        -1.74069782e-01]), -3317.020263671875, -100, array([ -328.86774111,  -331.37907043, -2734.19433345, -3317.19433345]), 0), (array([ -3.56094520e+00,   0.00000000e+00,   3.57000000e+02,
         3.13914945e-01]), -2033.4605712890625, -100, array([ -206.52061246,  -202.95966726, -1676.14665634, -2033.14665634]), 0), (array([  4.79214413e+00,   0.00000000e+00,   3.01000000e

...

[(array([  7.78637907e-01,   0.00000000e+00,   4.25000000e+02,
         2.76046127e-01]), -2419.376953125, -100, array([ -241.13535798,  -241.91399589, -1994.100907  , -2419.100907  ]), 0), (array([  4.05944982e+00,   0.00000000e+00,   4.17000000e+02,
        -1.31022201e-01]), -2371.1640625, -100, array([ -232.45498406,  -236.51443388, -1954.2950847 , -2371.2950847 ]), 0), (array([  9.26522035e-01,   0.00000000e+00,   4.83000000e+02,
        -1.77025140e-01]), -2747.84619140625, -100, array([ -273.82055323,  -274.74707527, -2265.02321655, -2748.02321655]), 0), (array([  7.57972581e-01,   0.00000000e+00,   5.93000000e+02,
        -2.15597092e-01]), -3374.324951171875, -100, array([ -336.64367314,  -337.40164572, -2781.54054826, -3374.54054826]), 0), (array([ -2.18691210e+00,   0.00000000e+00,   5.96000000e+02,
         2.75314980e-01]), -3394.304443359375, -100, array([ -341.37395133,  -339.18703923, -2798.02912838, -3394.02912838]), 0), (array([  1.13613763e+00,   0.00000000e+00,   5.

[(array([  -4.35345078,    0.        ,  440.        ,    0.44118347]), -2507.00830078125, -100, array([ -254.33853783,  -249.98508704, -2066.56711731, -2506.56711731]), 0), (array([   3.02879017,    0.        ,  448.        ,   -0.45537798]), -2546.711181640625, -100, array([ -251.28129687,  -254.31008704, -2099.16655962, -2547.16655962]), 0), (array([ -3.40658228e+00,   0.00000000e+00,   5.42000000e+02,
         4.74148348e-01]), -3087.885498046875, -100, array([ -311.65544732,  -308.24886504, -2545.4113497 , -3087.4113497 ]), 0), (array([  6.89089639e-01,   0.00000000e+00,   5.51000000e+02,
         8.07054816e-02]), -3136.325439453125, -100, array([ -312.91956909,  -313.60865873, -2585.24473397, -3136.24473397]), 0), (array([ -2.50189221e+00,   0.00000000e+00,   4.48000000e+02,
        -2.03669254e-01]), -2549.372802734375, -100, array([ -257.20052653,  -254.69863432, -2101.57647199, -2549.57647199]), 0), (array([   0.96129141,    0.        ,  298.        ,    0.31241103]), -1696.04

[(array([ -4.02525813e-01,   0.00000000e+00,   3.07000000e+02,
        -2.98069506e-01]), -1745.1942138671875, -100, array([ -174.91834294,  -174.51581713, -1438.49228337, -1745.49228337]), 0), (array([  4.25188556e+00,   0.00000000e+00,   5.92000000e+02,
        -1.48783454e-01]), -3367.8798828125, -100, array([ -331.60176241,  -335.85364796, -2776.02866627, -3368.02866627]), 0), (array([ -4.92241940e+00,   0.00000000e+00,   6.05000000e+02,
        -2.20138517e-01]), -3444.32763671875, -100, array([ -348.09342577,  -343.17100637, -2839.54777524, -3444.54777524]), 0), (array([  -3.37916032,    0.        ,  318.        ,    0.48811134]), -1811.9725341796875, -100, array([ -184.26726636,  -180.88810604, -1493.48442284, -1811.48442284]), 0), (array([ -4.03950681e-01,   0.00000000e+00,   5.74000000e+02,
         2.51824138e-01]), -3268.366943359375, -100, array([ -327.23322715,  -326.82927647, -2694.11511922, -3268.11511922]), 0), (array([ -4.88792841e+00,   0.00000000e+00,   5.01000000e+0

[(array([   2.83069124,    0.        ,  381.        ,   -0.46425436]), -2165.0830078125, -100, array([ -213.40849349,  -216.23918473, -1784.54726217, -2165.54726217]), 0), (array([ -1.69656826e+00,   0.00000000e+00,   4.00000000e+02,
         1.22268588e-01]), -2277.0615234375, -100, array([ -229.30586288,  -227.60929462, -1876.93925485, -2276.93925485]), 0), (array([  6.09535043e-01,   0.00000000e+00,   5.41000000e+02,
         2.92517710e-01]), -3080.2607421875, -100, array([ -307.39804503,  -308.00758007, -2538.96822448, -3079.96822448]), 0), (array([ -3.28868760e+00,   0.00000000e+00,   3.31000000e+02,
        -1.47215906e-01]), -1883.3740234375, -100, array([ -191.32166554,  -188.03297794, -1552.52123934, -1883.52123934]), 0), (array([  4.54765695e+00,   0.00000000e+00,   2.99000000e+02,
        -1.10284462e-02]), -1699.3572998046875, -100, array([ -164.84514046,  -169.39279741, -1400.36832825, -1699.36832825]), 0), (array([  4.97941044e+00,   0.00000000e+00,   5.11000000e+02,
   

KeyboardInterrupt: 

In [94]:
torch.Tensor(torch.randn([3,4]))


 0.1607 -1.2079 -0.6674  0.3834
 0.3433 -0.9077  0.1283 -0.2258
 0.2682 -1.3087  1.0791 -0.1893
[torch.FloatTensor of size 3x4]